1. Start with original dataset
2. Keep only data points where author date of birth is known
3. For each quote, compute author age at this time (quote date - author date of birth)
4. Compute means per decade (of age)
5. Regress opinion score on binary decade indicator variable
NB : we can repeat the process for age ranges of length 5 / 20 / 40… years
6. Test for coefficients and compare the coefficients’ signs
Conclude


In [49]:
import pandas as pd


In [53]:
data = pd.read_pickle("./added_sentiment.pkl")
data.drop(data[data.date_of_birth == ''].index, inplace=True)  # filter damaged data (1 raw)
data["age"] = data.quote_year.astype(int) - data.date_of_birth.astype(int)  # get age
data.head()

,date_of_birth,nationality,gender,occupation,Speaker,Quote,numOccurrences,quote_year,quote_month,Sentiment,age
0,1971,[United States of America],[female],[LGBTIQ+ rights activist],sarah kate ellis,This loss is a wake-up call that despite remar...,2,2015,11,-0.8760,44
1,1971,[United States of America],[female],[LGBTIQ+ rights activist],sarah kate ellis,"She didn't see it, she hadn't heard of it, she...",1,2015,6,0.0387,44
2,1971,[United States of America],[female],[LGBTIQ+ rights activist],sarah kate ellis,As a journalist and anchor who reaches million...,1,2015,4,0.4939,44
3,1971,[United States of America],[female],[LGBTIQ+ rights activist],sarah kate ellis,By empowering people to talk about their gende...,6,2015,2,0.7003,44
4,1971,[United States of America],[female],[LGBTIQ+ rights activist],sarah kate ellis,"By investing in this dangerous programming, TL...",133,2015,1,-0.5610,44


In [54]:
for col in ["nationality", "gender", "occupation"]:
    first = pd.Series([elem[0] if elem[0] is not None else "F" for elem in data[col]])  # keep 1st element
    data[col] = first.values
    data = data[data[col] != "F"]  # filter NaN (~10 raws)
data.head()

,date_of_birth,nationality,gender,occupation,Speaker,Quote,numOccurrences,quote_year,quote_month,Sentiment,age
0,1971,United States of America,female,LGBTIQ+ rights activist,sarah kate ellis,This loss is a wake-up call that despite remar...,2,2015,11,-0.8760,44
1,1971,United States of America,female,LGBTIQ+ rights activist,sarah kate ellis,"She didn't see it, she hadn't heard of it, she...",1,2015,6,0.0387,44
2,1971,United States of America,female,LGBTIQ+ rights activist,sarah kate ellis,As a journalist and anchor who reaches million...,1,2015,4,0.4939,44
3,1971,United States of America,female,LGBTIQ+ rights activist,sarah kate ellis,By empowering people to talk about their gende...,6,2015,2,0.7003,44
4,1971,United States of America,female,LGBTIQ+ rights activist,sarah kate ellis,"By investing in this dangerous programming, TL...",133,2015,1,-0.5610,44


In [55]:
# encode category data by indexes
nationality_dict = {elem: idx for idx, elem in enumerate(data.nationality.value_counts().index)}
gender_dict = {elem: idx for idx, elem in enumerate(data.gender.value_counts().index)}
occupation_dict = {elem: idx for idx, elem in enumerate(data.occupation.value_counts().index)}
data.replace({"nationality": nationality_dict, "gender": gender_dict, "occupation": occupation_dict}, inplace=True)
data.head()

{'United States of America': 0,
 'United Kingdom': 1,
 'Australia': 2,
 'Canada': 3,
 'India': 4,
 'Philippines': 5,
 'Ireland': 6,
 'New Zealand': 7,
 'Germany': 8,
 'South Africa': 9,
 'Israel': 10,
 'France': 11,
 'Italy': 12,
 'Brazil': 13,
 'United Kingdom of Great Britain and Ireland': 14,
 'Soviet Union': 15,
 'Pakistan': 16,
 'Nigeria': 17,
 'Sweden': 18,
 'England': 19,
 'Kingdom of the Netherlands': 20,
 'Mexico': 21,
 'Malaysia': 22,
 'Uganda': 23,
 'Poland': 24,
 "People's Republic of China": 25,
 'Japan': 26,
 'Scotland': 27,
 'Iran': 28,
 'Puerto Rico': 29,
 'Switzerland': 30,
 'Indonesia': 31,
 'Argentina': 32,
 'Russia': 33,
 'Jamaica': 34,
 'Belgium': 35,
 'Kenya': 36,
 'Chile': 37,
 'Denmark': 38,
 'Spain': 39,
 'Singapore': 40,
 'Great Britain': 41,
 'Taiwan': 42,
 'Austria': 43,
 'Ukraine': 44,
 'The Bahamas': 45,
 'South Korea': 46,
 'Lebanon': 47,
 'Canadians': 48,
 'Trinidad and Tobago': 49,
 'Kingdom of England': 50,
 'Norway': 51,
 'Egypt': 52,
 'Wales': 53,
 '

In [ ]:
# try sklearn ridge regression
y = data.Sentiment.to_numpy()
x = data.drop(["Sentiment", "Quote", "Speaker"], axis=1).to_numpy()
x[:, 0] = list(map(int, x[:, 0]))
# x = x[:, [0, -1]]  # take specific features
print(x.shape)

In [ ]:
from sklearn.linear_model import Ridge
reg = Ridge(8).fit(x, y)
print("Score:", reg.score(x, y))
print("Coefficients:", reg.coef_)
print("Intercept:", reg.intercept_)

In [57]:
# try statsmodels 
import statsmodels.formula.api as smf
data.date_of_birth = data.date_of_birth.apply(lambda k: int(k))
mod = smf.ols(formula='Sentiment ~ date_of_birth + nationality + gender + occupation + numOccurrences + quote_year + quote_month	+ age',
              data = data)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:              Sentiment   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     13.38
Date:                Thu, 16 Dec 2021   Prob (F-statistic):           2.25e-17
Time:                        09:57:42   Log-Likelihood:                -90768.
No. Observations:              114684   AIC:                         1.816e+05
Df Residuals:                  114676   BIC:                         1.816e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          5.0719      2.179      2.